In [1]:
import os
import cv2
import sys
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

In [ ]:
def process_image(image_path, umperpx_path, output_dir, current_sample, expansion_distances_microns):
    os.makedirs(output_dir, exist_ok=True)
    print(f"Current working directory: {os.getcwd()}")
    if not os.path.exists(umperpx_path):
        sys.exit(f"File not found: {umperpx_path}")

    umperpx_df = pd.read_csv(umperpx_path, sep='\t', header=None, names=['Sample', 'Axis', 'Value'])
    Axis_value = umperpx_df[(umperpx_df['Sample'] == current_sample)]['Value'].values
    if len(Axis_value) == 0:
        sys.exit(f"Could not find Axis value for sample {current_sample}.")
    Axis_value = Axis_value[0]

    image = cv2.imread(image_path)
    if image is None:
        sys.exit('Could not read the image.')
    height, width = image.shape[:2]

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)
    kernel = np.ones((3, 3), np.uint8)
    edges = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    boundary_image = np.zeros_like(image)
    cv2.drawContours(boundary_image, contours, -1, (0, 255, 0), 1)
    cv2.imwrite(os.path.join(output_dir, "boundary_image.png"), boundary_image)

    expanded_boundary_images = []

    for distance_microns in expansion_distances_microns:
        distance_pixels = int(distance_microns / Axis_value)
        kernel_size = distance_pixels * 2 + 1
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
        expanded_boundary_image = np.zeros_like(image)
        for contour in contours:
            single_contour_image = np.zeros_like(image)
            cv2.drawContours(single_contour_image, [contour], -1, (255, 255, 255), 1)
            expanded_contour_out = cv2.dilate(single_contour_image, kernel, iterations=1)
            expanded_boundary_image = cv2.bitwise_or(expanded_boundary_image, expanded_contour_out)
        expanded_boundary_image[expanded_boundary_image > 0] = 255
        expanded_boundary_images.append(expanded_boundary_image)
        cv2.imwrite(os.path.join(output_dir, f"expanded_boundary_{distance_microns}um.png"), expanded_boundary_image)
    result_image = image.copy()

    for i, expanded_boundary_image in enumerate(expanded_boundary_images):
        expanded_contours, _ = cv2.findContours(cv2.cvtColor(expanded_boundary_image, cv2.COLOR_BGR2GRAY), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(result_image, expanded_contours, -1, (0, 0, 0), 1)

    cv2.imwrite(os.path.join(output_dir, "result_image.png"), result_image)

    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
    plt.title('Result Image with Expanded Boundaries')
    plt.axis('off')
    plt.show()


In [ ]:
input_dir = '/data/work/pathologicalRegion/boundaryExpansion/boundary_plot'
#1000×1000 pixel image files in the input_dir directory,you can use Analysis/Pathological_regions/01.inputImage.ipynb to generate these images
umperpx_path = '/data/work/pathologicalRegion/boundaryExpansion/bin100-umperpx.txt'
#you can use 01.caluculate_per_px.R to generate this file
output_base_dir = '/data/work/pathologicalRegion/boundaryExpansion/output'
csv_path = '/data/work/pathologicalRegion/boundaryExpansion/limitation.csv'
#Contains two columns: 'filename' (image names without extensions in input_dir) and 'uplimit' (maximum boundary extension distance)


uplimit_df = pd.read_csv(csv_path)
uplimit_df.columns = ['filename', 'uplimit']

image_paths = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.png')]

for image_path in image_paths:
    image_name = os.path.splitext(os.path.basename(image_path))[0]
    #current_sample = '_'.join(image_name.split('_')[:2])
    current_sample = image_name.split('.')[0]
    uplimits = uplimit_df[uplimit_df['filename'] == image_name]['uplimit'].tolist()
    for uplimit in uplimits:
        expansion_distances_microns = list(range(100, uplimit + 1, 100))
        
        output_dir = os.path.join(output_base_dir, f"{image_name}_{uplimit}")
        process_image(image_path, umperpx_path, output_dir, current_sample, expansion_distances_microns)

Current working directory: /data/work/pathologicalRegion/boundaryExpansion


In [ ]:
print('ggg')